### Tratamento das imagens em diferentes camadas

In [ ]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(300,300,3)),

    layers.Resizing(300,300),
    layers.Rescaling(1./255),

    layers.Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same'),           
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(2, activation=activations.softmax, kernel_initializer=initializers.RandomNormal())
])

### Configurando o optimizer

In [ ]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.binary_accuracy ]
)

### Selecionando as imagens para o treinamento e teste

In [ ]:
from tensorflow.keras import utils

path = './CatsVsDogs'
batch_size = 64

train = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    seed = 1,
    image_size = (300,300),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    seed = 1,
    image_size = (300,300),
    batch_size = batch_size
)

### Realizando o treinamento do modelo

In [ ]:
from tensorflow.keras import callbacks

model_path = "./catsvsdogs.keras"

patience = 5
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)